In [1]:
import requests, time, os
from datetime import datetime
import xml.etree.ElementTree as ET
import xml_to_df
import pandas as pd

In [3]:
ids_pvp = pd.read_csv("data/ListaProposicoesVotadasPlenario_2013_2024.csv") 

In [4]:
# example

url_vpi = 'https://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ObterVotacaoProposicaoPorID'
id = 1672576
params = {
    'idProposicao': id
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [5]:
response_test = requests.get(url_vpi, params=params, headers=headers)

In [6]:
root = ET.fromstring(response_test.text)
xml_text = ET.tostring(root, encoding='unicode')
print(xml_text)

<proposicao>
  <Votacoes>
    <Votacao Resumo="Rejeitado o Requerimento. Sim: 27; não: 394; total: 421.&#13;&#10;" Data="28/8/2024" Hora="18:41" ObjVotacao="REQUERIMENTO DE RETIRADA DE PAUTA" codSessao="17500" nomeProposicao="PL 1829/2019 (Nº Anterior: PL 2724/2015)">
      <orientacaoBancada>
        <bancada Sigla="Bl UniPpFdrPsdbCid..." orientacao="Não            " />
        <bancada Sigla="Bl MdbPsdRepPode" orientacao="Não            " />
        <bancada Sigla="PL" orientacao="Não            " />
        <bancada Sigla="Fdr PT-PCdoB-PV" orientacao="Não            " />
        <bancada Sigla="PSB" orientacao="Não            " />
        <bancada Sigla="Fdr PSOL-REDE" orientacao="Sim            " />
        <bancada Sigla="Repr.Novo" orientacao="Sim            " />
        <bancada Sigla="Maioria" orientacao="Não            " />
        <bancada Sigla="Minoria" orientacao="Liberado       " />
        <bancada Sigla="Oposição" orientacao="Liberado       " />
        <bancada Sigla="

In [25]:
responses_vpi = {}
dfs_vpi = {}

lista_dados = []

for id in ids_pvp.codProposicao.unique():
    
    params = {
        'idProposicao': id
    }

    try:
        response = xml_to_df.get_data(url_vpi, params, headers)
        root = ET.fromstring(response.text)

        for votacao in root.findall(".//Votacao"):
            dados = {}
            for attr in votacao.attrib:
                dados[f'{attr.lower()}'] = votacao.get(attr)
            
            # Iterar sobre orientações de bancadas
            for bancada in votacao.findall(".//bancada"):
                for attr in bancada.attrib:
                    dados[f'bancada_{attr.lower()}'] = bancada.get(attr)

            # Dicionário para armazenar a quantidade de deputados por partido
            dados_partidos = {}

            # Contagem de deputados por partido
            for deputado in votacao.findall(".//Deputado"):
                partido = deputado.get('Partido').strip()  # Remove espaços ao redor do nome do partido
                
                # Se o partido ainda não estiver no dicionário, inicialize com zero
                if partido not in dados_partidos:
                    dados_partidos[partido] = 0
                
                # Incrementa a quantidade de deputados do partido
                dados_partidos[partido] += 1
            
            # Calcula o total de deputados
            total_deputados = sum(dados_partidos.values())

            # Dicionário para armazenar o percentual de deputados por partido
            percentual_partidos = {}

            # Calcula e armazena o percentual de deputados por partido
            for partido, quantidade in dados_partidos.items():
                percentual = (quantidade / total_deputados) * 100
                percentual_partidos[partido] = percentual

            # Adiciona os percentuais ao dicionário de dados
            for partido, percentual in percentual_partidos.items():
                dados[f'percentual_{partido}'] = percentual

            lista_dados.append(dados)

        if response.status_code != 200:
            print(f'Erro ao extrair dados de votação do PL de id = {id}')
            continue
        
        df = pd.DataFrame(lista_dados)
        dfs_vpi[id] = df
        responses_vpi[id] = response
    
    except Exception as e:
        print(f'Erro ao processar dados de votação do PL de id = {id}: {e}')

merged_df = pd.concat(dfs_vpi.values(), ignore_index=True)
xml_to_df.save_csv(merged_df, 'votacoes_proposicoes_2013_2024.csv')


In [8]:
response = requests.get(url_vpi, params=params, headers=headers)
root = ET.fromstring(response.text)

In [24]:
# Dicionário para armazenar a quantidade de deputados por partido
dados_partidos = {}

# Contagem de deputados por partido
for votacao in root.findall(".//Deputado"):
    partido = votacao.get('Partido').strip()  # Remove espaços ao redor do nome do partido
    
    # Se o partido ainda não estiver no dicionário, inicialize com zero
    if partido not in dados_partidos:
        dados_partidos[partido] = 0
    
    # Incrementa a quantidade de deputados do partido
    dados_partidos[partido] += 1

# Calcula o total de deputados
total_deputados = sum(dados_partidos.values())

# Dicionário para armazenar o percentual de deputados por partido
percentual_partidos = {}

# Calcula e armazena o percentual de deputados por partido
for partido, quantidade in dados_partidos.items():
    percentual = (quantidade / total_deputados) * 100
    percentual_partidos[partido] = percentual

# Exibe o dicionário com os percentuais
print(percentual_partidos)


{'PMDB': 17.328519855595665, 'PR': 6.4981949458483745, 'PT': 20.938628158844764, 'PCdoB': 4.332129963898916, 'Solidaried': 2.527075812274368, 'PPS': 1.083032490974729, 'PTB': 2.527075812274368, 'PSD': 7.581227436823104, 'DEM': 3.2490974729241873, 'PV': 1.8050541516245486, 'PDT': 5.415162454873646, 'PP': 5.415162454873646, 'PSDB': 6.4981949458483745, 'PROS': 4.693140794223827, 'PSB': 5.776173285198556, 'PTdoB': 0.36101083032490977, 'PRB': 1.444043321299639, 'PSOL': 1.083032490974729, 'PSC': 0.36101083032490977, 'PMN': 0.7220216606498195, 'PSDC': 0.36101083032490977}


In [22]:
dados_partidos

{'PMDB': {'Votos_Sim': 0, 'Votos_Nao': 46},
 'PR': {'Votos_Sim': 0, 'Votos_Nao': 16},
 'PT': {'Votos_Sim': 0, 'Votos_Nao': 55},
 'PCdoB': {'Votos_Sim': 0, 'Votos_Nao': 12},
 'Solidaried': {'Votos_Sim': 0, 'Votos_Nao': 2},
 'PPS': {'Votos_Sim': 0, 'Votos_Nao': 0},
 'PTB': {'Votos_Sim': 1, 'Votos_Nao': 6},
 'PSD': {'Votos_Sim': 1, 'Votos_Nao': 4},
 'DEM': {'Votos_Sim': 0, 'Votos_Nao': 0},
 'PV': {'Votos_Sim': 0, 'Votos_Nao': 5},
 'PDT': {'Votos_Sim': 0, 'Votos_Nao': 14},
 'PP': {'Votos_Sim': 1, 'Votos_Nao': 11},
 'PSDB': {'Votos_Sim': 0, 'Votos_Nao': 0},
 'PROS': {'Votos_Sim': 0, 'Votos_Nao': 13},
 'PSB': {'Votos_Sim': 0, 'Votos_Nao': 15},
 'PTdoB': {'Votos_Sim': 0, 'Votos_Nao': 1},
 'PRB': {'Votos_Sim': 0, 'Votos_Nao': 4},
 'PSOL': {'Votos_Sim': 0, 'Votos_Nao': 3},
 'PSC': {'Votos_Sim': 0, 'Votos_Nao': 0},
 'PMN': {'Votos_Sim': 0, 'Votos_Nao': 2},
 'PSDC': {'Votos_Sim': 0, 'Votos_Nao': 1}}

In [23]:
len(dados_partidos)

21

In [ ]:
 for votacao in root.findall(".//Votacao"):
    for attr in votacao.attrib:
        dados[f'{attr.lower()}'] = votacao.get(attr)
            
            # Iterar sobre orientações de bancadas
    for bancada in votacao.findall(".//bancada"):
                for attr in bancada.attrib:
                    dados[f'bancada_{attr.lower()}'] = bancada.get(attr)

            # Iterar sobre os votos dos deputados
    for deputado in votacao.findall(".//Deputado"):
                for attr in deputado.attrib:
                    dados[f'deputado_{attr.lower()}'] = deputado.get(attr)